# Симплициальные многогранники и внешние формы

## Порядок сдачи
Срок сдачи задания -- **1 апреля**. 2 апреля работы будут собраны и уйдут на проверку. Сдача после дедлайна возможна в индивидуальном порядке за половинный балл.

Можете объединяться в команды по 2-3 человека для совместного написания программ:
для этого нужно пригласить соответствующего студента в коллабораторы и указать в комментариях к коду, кто реализовал какое задание.
Можете реализовывать задания по своему выбору. 
Максимальная сумма, присуждаемая одному участнику -- **30** баллов.
Если команда (из 1 или более человека) корректно реализовала все методы -- дополнительно присуждается по **10** баллов каждому.

Будьте готовы к тому, что мы попросим некоторых из вас объяснить, что делает программа и как работает код. Использование чужого кода наказуемо аннулированием баллов у всех сторон подобного инцидента.
По всем вопросам пишите в чат в правом верхнем углу - мы увидим оповещение и ответим. Вопросы общего характера можно также задавать в telegram-чате.

## Соглашения

Мы задаём ориентируемое симплициальное многообразие X без края с помощью класса Variety. Оно обладает X.n вершинами, пронумерованными числами от 0 до n-1.
Рёбра задаются парами вершин, (полу)рёбра -- парами вершин с учётом их порядка, а грани задаются тройками вершин с учётом ориентации, то есть направления обхода вершин.
Основным полем объекта X является X.faces -- массив ориентированных граней.
- *0-форма* (функция на вершинах) является функцией, принимающей номер вершины и возвращающей значение.
- *1-форма* (функция на рёбрах) является функцией, принимающей пару номеров вершин и возвращающей значение. Если пара вершин не задаёт ребро, то функция вольна возвращать произвольное значение. 
- *2-форма* (функция на гранях) является функцией, принимающей тройку номеров вершин и возвращающей значение. Если тройка вершин не задаёт грань, то функция вольна возвращать произвольное значение. 

## Задания

1. *(5 баллов)* Напишите метод **check**(), проверяющий, задаёт ли faces (массив троек номеров вершин) симплициальное многообразие (ориентируемое, без края), и возвращающий соответствующее булево значение. Вам нужно проверить, что окрестность каждой вершины и окрестность каждого ребра гомеоморфны плоскости. *Указание:* удобнее проверять на полурёбрах.
2. *(5 баллов)* Напишите метод **Euler**(), возвращающий эйлерову характеристику симплициального многообразия.
3. *(5 баллов)* Напишите метод **d_0**(form), принимающий 0-форму и возвращающий её дифференциал -- 1-форму.
4. *(5 баллов)* Напишите метод **check_form**(k, form), принимающий на вход k-форму и проверяющий её корректность, где k = 1 или 2.
5. *(5 баллов)* Напишите метод **d_1**(form), принимающий 1-форму и возвращающий её дифференциал -- 2-форму.
6. Напишите метод **wedge**(k1,k2, form1, form2), принимающий k1-форму и k2-форму и возвращающий их внешнее произведение, для (k1,k2)=
  - *(5 баллов)* (0,0).
  - *(5 баллов)* (0,1) и (1,0).
  - *(10 баллов)* (0,2) и (2,0).
  - *(15 баллов)* (1,1).


In [21]:
import numpy as np
from numpy.linalg import det
import itertools
from sympy.combinatorics import Permutation

In [15]:
list(itertools.permutations([1, 2, 3]))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [27]:
p = Permutation((0, 1, 2))
p.array_form

[0, 1, 2]

In [73]:
class Variety():
    def __init__(self, faces):
        self.faces = faces
        vertices = set(i for f in faces for i in f)
        self.n = max(vertices)+1
        assert set(range(self.n)) == vertices
        self.edges = self._get_edges(return_unique=False)
        self.unique_edges = self._get_edges(return_unique=True)
        for f in faces:
            assert len(f)==3

    @property
    def num_vertices(self):
        return self.n

    def check(self):
        pass # Замените 'pass' на код функции, проверяющий self.faces

    def Euler(self):
        return self.num_vertices - len(self.unique_edges) + len(self.faces)

    def d_0(self, func):
        return lambda x, y: func(y) - func(x)

    def d_1(self, func):
        return lambda x, y, z: func(x, y) + func(y, z) + func(z, x)

    def check_form(self, k, func):
        assert k in [1, 2]
        if k == 1:
            if False in [func(edge[0], edge[1]) + func(edge[1], edge[0]) == 0 for edge in self.unique_edges]:
                return False
            return True
        else:
            for face in self.faces:
                face = np.array(face)
                if False in [func(*face) == Permutation(perm).signature() * func(*face[list(perm)]) for perm in itertools.permutations([0, 1, 2])]:
                    return False
            return True
            

    def wedge(self, k1, k2, f1, f2):
        assert k1 in [0, 1, 2] and k2 in [0, 1, 2]
        if k1 == k2 == 0:
            return lambda x: f1(x) * f2(x)
        if k1 == 0 and k2 == 1:
            return lambda x, y: f2(x, y) * (f1(x) + f1(y)) / 2
        if k1 == 1 and k2 == 0:
            return lambda x, y: f1(x, y) * (f2(x) + f2(y)) / 2
        if k1 == 0 and k2 == 2:
            return lambda x, y, z: (f1(x) + f1(y) + f1(z)) / 3 * f2(x, y, z)
        if k2 == 0 and k1 == 2:
            return lambda x, y, z: (f2(x) + f2(y) + f2(z)) / 3 * f1(x, y, z)
        if k1 == 1 and k2 == 1:
            return lambda x, y, z: 1 / 6 * det(
                                                np.array([
                                                            [1, 1, 1],
                                                            [f1(x, y), f1(y, z), f1(z, x)],
                                                            [f2(x, y), f2(y, z), f2(z, x)]
                                                        ])
                                               )

    def _get_edges(self, return_unique):
        edges = []
        for i, j, k in self.faces:
            edges.append((i, j))
            edges.append((j, k))
            edges.append((k, i))
        if return_unique:
            return list(set([tuple(sorted(x)) for x in edges]))
        else:
            return edges

In [74]:
# примеры проверок (сначала запускаете ячейку с классом, потом с проверкой):
sphere = Variety([(3,2,1), (2,3,0), (1,0,3),(0,1,2)])
torus = Variety([
        (1,0,3),
        (1,3,2),
        (2,3,6),
        (3,4,6),
        (4,0,6),
        (1,6,0),
        (2,6,5),
        (1,5,6),
        (2,5,0),
        (3,0,5),
        (5,4,3),
        (1,4,5),
        (1,2,4),
        (2,0,4),
    ])

def test1():
    assert sphere.check() == True
    assert torus.check() == True
    assert Variety([(1,2,3), (2,3,0), (3,0,1),(0,1,2)]).check() == False
    assert Variety([(1,2,3), (2,3,1), (3,0,1),(0,1,2)]).check() == False
    assert Variety([(1,2,0), (1,0,2)]).check() == False
    assert Variety([(3,2,1), (2,3,0), (1,0,3),(0,1,2), (6,5,4), (5,6,0), (4,0,6),(0,4,5)]).check() == False


def test2():
    assert sphere.Euler() == 2
    assert torus.Euler() == 0

def test3():
    assert sphere.d_0(lambda x : x)(1, 2) == 1
    assert torus.d_0(lambda x : x**2)(4, 3) == -7

def test4():
    assert sphere.check_form(2, lambda x,y,z : x+y+z if (x-y)*(y-z)*(z-x)>0 else -(x+y+z)) == True
    assert torus.check_form(1,lambda v,w : 1) == False

def test5():
    assert sphere.d_1(lambda x, y : x-y)(0, 1, 2) == 0
    assert sphere.d_1(lambda x, y : x*y if x<y else -x*y)(1, 2, 3) == 5

def test61():
    assert sphere.wedge(0,0, lambda x: x, lambda x: -x)(2) == -4

def test62():
    assert sphere.wedge(0,1, lambda x: x, lambda x,y: y-x)(2,3) == 5/2

def test63():
    assert sphere.wedge(0,2, lambda x: x, lambda x,y,z:  x+y+z if (x-y)*(y-z)*(z-x)>0 else -(x+y+z) )(0,1,2) == 3

def test64():
    assert sphere.wedge(1,1, lambda x,y: x-y, lambda x,y: y-x)(1,2,3) == 0

tests = [(1, test1),
         (2, test2),
         (3, test3),
         (4, test4),
         (5, test5),
         ("6.1", test61),
         ("6.2", test62),
         ("6.3", test63),
         ("6.4", test64),
        ]

for i, t in tests:
    try:
        t()
    except:
        print('Функция задания {0} не прошла проверку'.format(i))
    else:
        print('Ошибок в задании {} не найдено'.format(i))



Функция задания 1 не прошла проверку
Ошибок в задании 2 не найдено
Ошибок в задании 3 не найдено
Ошибок в задании 4 не найдено
Ошибок в задании 5 не найдено
Ошибок в задании 6.1 не найдено
Ошибок в задании 6.2 не найдено
Ошибок в задании 6.3 не найдено
Ошибок в задании 6.4 не найдено
